# **Data understanding and clening.**

# Downloading Dataset.

In [8]:
!mkdir ~/data
!curl -L -o ~/cryptocurrencypricehistory.zip\
  https://www.kaggle.com/api/v1/datasets/download/sudalairajkumar/cryptocurrencypricehistory
!unzip ~/cryptocurrencypricehistory.zip -d ~/data

mkdir: cannot create directory ‘/root/data’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1740k  100 1740k    0     0  4283k      0 --:--:-- --:--:-- --:--:-- 4283k
Archive:  /root/cryptocurrencypricehistory.zip
replace /root/data/coin_Aave.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /root/data/coin_Aave.csv  
replace /root/data/coin_BinanceCoin.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /root/data/coin_BinanceCoin.csv  
replace /root/data/coin_Bitcoin.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /root/data/coin_Bitcoin.csv  
replace /root/data/coin_Cardano.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /root/data/coin_Cardano.csv  
replace /root/data/coin_ChainLink.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /root/

In [2]:
import pywt
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates  # Для работы с датами на графиках

from datetime import datetime
from statsmodels.tsa.stattools import acf, pacf  # ACF и PACF

In [3]:
"""
Загружает и очищает данные о криптовалютах с Yahoo Finance.

Args:
    tickers (list): Список тикеров криптовалют (например, ['BTC-USD', 'ETH-USD']).
    start_date (str): Начальная дата в формате 'YYYY-MM-DD'.
    end_date (str): Конечная дата в формате 'YYYY-MM-DD'.

Returns:
    dict: Словарь, где ключи - тикеры, а значения - DataFrame с очищенными данными.
        Если данные для тикера не удалось загрузить, значение будет None.
"""
def load_and_clean_data(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        try:
            # Загрузка данных с yfinance
            df = yf.download(ticker, start=start_date, end=end_date)

            # Проверка на наличие данных
            if df.empty:
                print(f"Данные для {ticker} не найдены.")
                data[ticker] = None
                continue

            # Проверка типов данных и преобразование при необходимости
            for col in ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']:
                if df[col].dtype == 'object':  # Если вдруг строка
                    try:
                        df[col] = pd.to_numeric(df[col], errors='coerce') # errors='coerce' заменит нечисловые значения на NaN
                    except ValueError:
                        print(f"Ошибка преобразования данных в числовой формат для {ticker}, столбец {col}.")
                        data[ticker] = None  # Или другое действие по обработке ошибки
                        continue

            # Обработка пропущенных значений (NaN)
            # Вариант 1: Удаление строк с NaN (если их немного)
            # df = df.dropna()

            # Вариант 2: Заполнение пропусков (интерполяция)
            df = df.interpolate(method='linear') # Линейная интерполяция
            # Другие методы интерполяции: 'time', 'nearest', 'spline', 'polynomial'
            # Выбор метода зависит от характера данных.  Для финансовых данных часто подходит линейная.

            # Вариант 3: Заполнение предыдущим значением (forward fill)
            # df = df.ffill()
            # Вариант 4: Заполнение константой
            #df = df.fillna(0)  # Не лучший выбор для финансовых данных

            # Проверка на наличие дубликатов по дате
            if df.index.duplicated().any():
                print(f"В данных для {ticker} обнаружены дубликаты по дате.  Удаляем дубликаты.")
                df = df[~df.index.duplicated(keep='first')] # Оставляем первое вхождение

            data[ticker] = df

        except Exception as e:
            print(f"Ошибка при загрузке данных для {ticker}: {e}")
            data[ticker] = None

    return data


In [4]:
def visualize_data(data):
    """
    Визуализирует данные о криптовалютах.

    Args:
        data (dict): Словарь, возвращаемый функцией load_and_clean_data.
    """

    for ticker, df in data.items():
        if df is not None:
            # Создание фигуры и набора осей
            fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14, 8), sharex=True)

            # График цен (Close)
            axes[0].plot(df.index, df['Close'], label=f'{ticker} Close Price', color='blue')
            axes[0].set_title(f'{ticker} - Close Price and Volume')
            axes[0].set_ylabel('Price (USD)')
            axes[0].grid(True)
            axes[0].legend()

             # Установка формата даты на оси X
            axes[0].xaxis.set_major_locator(mdates.AutoDateLocator())  # Автоматический выбор расположения меток
            axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) # Формат даты
            fig.autofmt_xdate() # Наклон меток для лучшей читаемости


            # График объема торгов
            axes[1].bar(df.index, df['Volume'], label=f'{ticker} Volume', color='gray', alpha=0.7)
            axes[1].set_ylabel('Volume')
            axes[1].grid(True)
            axes[1].legend()

            # Установка формата даты на оси X
            axes[1].xaxis.set_major_locator(mdates.AutoDateLocator())
            axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

            # График логарифма цены (Close) - добавляем, как обсуждали
            plt.figure(figsize=(14, 4))  # Отдельный график для логарифма
            plt.plot(df.index, np.log(df['Close']), label=f'{ticker} Log(Close Price)', color='green')
            plt.title(f'{ticker} - Logarithm of Close Price')
            plt.ylabel('Log(Price)')
            plt.grid(True)
            plt.legend()
            plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator()) # Авто-локатор
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Формат
            plt.gcf().autofmt_xdate() # Наклон

            # Отображение графиков
            plt.tight_layout()  # Чтобы графики не перекрывались
            plt.show()

In [5]:
def decompose_timeseries(data, window_size=30):
    """
    Выполняет первичную декомпозицию временных рядов с использованием скользящего среднего.

    Args:
        data (dict): Словарь, возвращаемый функцией load_and_clean_data.
        window_size (int): Размер окна для скользящего среднего.

    Returns:
        dict: Словарь, содержащий исходные данные, тренд, остатки, ACF и PACF остатков.
    """

    decomposed_data = {}

    for ticker, df in data.items():
        if df is not None:
            # Рассчитываем скользящее среднее (тренд)
            df['MA'] = df['Close'].rolling(window=window_size, center=True).mean()

            # Рассчитываем остатки (колебательная компонента)
            df['Residuals'] = df['Close'] - df['MA']

            # Удаляем NaN, появившиеся из-за скользящего среднего (по краям ряда)
            df_clean = df.dropna()  # Важно!  Иначе acf/pacf не будут работать

            # Рассчитываем ACF и PACF для остатков
            lags = min(len(df_clean) // 2 - 1, 40) #кол-во лагов, но не больше половины длины и не больше 40
            acf_values = acf(df_clean['Residuals'], nlags=lags)
            pacf_values = pacf(df_clean['Residuals'], nlags=lags)

            decomposed_data[ticker] = {
                'original': df,  # Исходный DataFrame с MA и Residuals
                'trend': df_clean['MA'],
                'residuals': df_clean['Residuals'],
                'acf': acf_values,
                'pacf': pacf_values
            }
    return decomposed_data


def visualize_decomposition(decomposed_data):
    """Визуализирует результаты декомпозиции."""
    for ticker, data in decomposed_data.items():
      if data is not None:
        df = data['original']

        fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

        # 1. Исходный ряд и тренд
        axes[0].plot(df.index, df['Close'], label='Original', color='blue')
        axes[0].plot(df.index, df['MA'], label=f'MA ({df["MA"].notna().sum()})', color='red') #label с указанием количества значений
        axes[0].set_title(f'{ticker} - Original Series and Moving Average (Trend)')
        axes[0].set_ylabel('Price (USD)')
        axes[0].grid(True)
        axes[0].legend()
        axes[0].xaxis.set_major_locator(mdates.AutoDateLocator())
        axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))


        # 2. Остатки
        axes[1].plot(df.index, df['Residuals'], label='Residuals', color='green')
        axes[1].set_title(f'{ticker} - Residuals')
        axes[1].set_ylabel('Residuals')
        axes[1].grid(True)
        axes[1].legend()
        axes[1].xaxis.set_major_locator(mdates.AutoDateLocator())
        axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

        # 3. ACF остатков
        axes[2].stem(np.arange(len(data['acf'])), data['acf'], markerfmt=" ", basefmt="-b") #Изменения тут
        axes[2].set_title(f'{ticker} - Autocorrelation Function (ACF) of Residuals')
        axes[2].set_xlabel('Lag')
        axes[2].set_ylabel('ACF')
        axes[2].grid(True)
        axes[2].axhline(y=0, color='k', linestyle='--') # Добавим линию 0
        axes[2].axhline(y=-1.96/np.sqrt(len(df['Residuals'].dropna())), color='gray', linestyle='--') # Добавим дов. интервалы
        axes[2].axhline(y=1.96/np.sqrt(len(df['Residuals'].dropna())), color='gray', linestyle='--')

        # 4. PACF остатков
        axes[3].stem(np.arange(len(data['pacf'])), data['pacf'], markerfmt=" ", basefmt="-b") #Изменения тут
        axes[3].set_title(f'{ticker} - Partial Autocorrelation Function (PACF) of Residuals')
        axes[3].set_xlabel('Lag')
        axes[3].set_ylabel('PACF')
        axes[3].grid(True)
        axes[3].axhline(y=0, color='k', linestyle='--') # Добавим линию 0
        axes[3].axhline(y=-1.96/np.sqrt(len(df['Residuals'].dropna())), color='gray', linestyle='--') # Добавим дов. интервалы
        axes[3].axhline(y=1.96/np.sqrt(len(df['Residuals'].dropna())), color='gray', linestyle='--')


        fig.autofmt_xdate()
        plt.tight_layout()
        plt.show()

In [7]:
tickers = ['BTC-USD', 'ETH-USD', 'ADA-USD', 'DOT-USD', 'BNB-USD']
start_date = '2020-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

crypto_data = load_and_clean_data(tickers, start_date, end_date)

# Вывод информации о загруженных данных
for ticker, df in crypto_data.items():
    if df is not None:
        print(f"\nДанные для {ticker}:")
        print(df.head())
        print(df.info())
        print(df.describe()) # Основные статистики
    else:
        print(f"\nДанные {ticker} не удалось загрузить и обработать.")

# Визуализация данных
visualize_data(crypto_data)

# Декомпозиция временных рядов
decomposed_crypto_data = decompose_timeseries(crypto_data, window_size=30)

# Визуализация результатов декомпозиции
visualize_decomposition(decomposed_crypto_data)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Ошибка при загрузке данных для BTC-USD: 'DataFrame' object has no attribute 'dtype'


Ошибка при загрузке данных для ETH-USD: 'DataFrame' object has no attribute 'dtype'


[*********************100%***********************]  1 of 1 completed


Ошибка при загрузке данных для ADA-USD: 'DataFrame' object has no attribute 'dtype'


[*********************100%***********************]  1 of 1 completed


Ошибка при загрузке данных для DOT-USD: 'DataFrame' object has no attribute 'dtype'


[*********************100%***********************]  1 of 1 completed

Ошибка при загрузке данных для BNB-USD: 'DataFrame' object has no attribute 'dtype'

Данные BTC-USD не удалось загрузить и обработать.

Данные ETH-USD не удалось загрузить и обработать.

Данные ADA-USD не удалось загрузить и обработать.

Данные DOT-USD не удалось загрузить и обработать.

Данные BNB-USD не удалось загрузить и обработать.


In [ ]:

# ... (Предыдущий код: load_and_clean_data, visualize_data, decompose_timeseries)

def cwt_analysis(data, wavelet_name='cmor1.5-1.0', scales=None, coif=None): #добавляем coif для отрисовки
    """
    Выполняет непрерывное вейвлет-преобразование (CWT) и визуализирует скалограмму.

    Args:
        data (pd.Series): Временной ряд для анализа (например, остатки).
        wavelet_name (str): Имя вейвлета (см. документацию PyWavelets).
                             Примеры: 'morl', 'gaus2', 'shan', 'cmorB-C' (B - полоса, C - центр. частота)
        scales (np.ndarray): Массив масштабов для CWT.  Если None, генерируется автоматически.
        coif(str): имя вейвлета для отрисовки самого вейвлета.

    Returns:
        tuple: (coef, frequencies) - вейвлет-коэффициенты и соответствующие частоты.
    """

    # 1. Определение масштабов
    if scales is None:
      #  scales = np.arange(1, 128) # Пример: линейный набор масштабов
        # Более разумный выбор для финансовых данных - логарифмический:
        scales = np.logspace(np.log10(2), np.log10(len(data)/2), num=100) #logspace - логарифмически-равномерная шкала

    # 2. Выполнение CWT
    coef, frequencies = pywt.cwt(data, scales, wavelet_name, sampling_period=1) #sampling_period=1, т.к. у нас шаг = 1 день

    # 3. Визуализация скалограммы
    plt.figure(figsize=(14, 6))
    plt.imshow(abs(coef), extent=[0, len(data), frequencies[-1], frequencies[0]], aspect='auto',
               cmap='jet', origin='upper') #origin='upper' для правильной ориентации
    plt.colorbar(label='Magnitude')
    plt.title(f'Scalogram ({wavelet_name})')
    plt.ylabel('Frequency (1/days)')  # Частота в обратных днях
    plt.xlabel('Time (days)')
    plt.gca().invert_yaxis() #перевернем ось Y

    #Отрисовка самого вейвлета
    if coif is not None:
        wavelet = pywt.Wavelet(coif)
        if wavelet.complex_cwt: #проверка на комплексность вейвлета
            [phi, x] = wavelet.wavefun(level=10)
            plt.figure(figsize=(14,3))
            plt.plot(x, np.real(phi), label='Real')
            plt.plot(x, np.imag(phi), label='Imaginary')
            plt.title(f'Wavelet: {coif}')
            plt.legend()
        else:
            [phi, psi, x] = wavelet.wavefun(level=10)
            plt.figure(figsize=(14,3))
            plt.plot(x,psi)
            plt.title(f'Wavelet: {coif}')

    plt.show()

    return coef, frequencies


# Пример использования:

# Выберем одну криптовалюту и остатки из предыдущей декомпозиции
ticker = 'BTC-USD'
residuals = decomposed_crypto_data[ticker]['residuals']

# Примеры использования с разными вейвлетами:
# 1. Complex Morlet (cmorB-C)
coef_cmor, frequencies_cmor = cwt_analysis(residuals, wavelet_name='cmor1.5-1.0', coif='cmor1.5-1.0') #B=1.5, C=1.0 - параметры

# 2. Morlet (morl)
coef_morl, frequencies_morl = cwt_analysis(residuals, wavelet_name='morl', coif='morl')

# 3. Gaussian derivative (gaus2) - вторая производная Гаусса
coef_gaus, frequencies_gaus = cwt_analysis(residuals, wavelet_name='gaus2', coif='gaus2')

# 4. Shannon (shan) - вейвлет Шеннона.  Пример: shanB-C, где B - полоса, C - центр. частота
coef_shan, frequencies_shan = cwt_analysis(residuals, wavelet_name='shan1-1.5', coif='shan1-1.5')

# 5. Complex Gaussian derivative
coef_cgau, frequencies_cgau = cwt_analysis(residuals, wavelet_name = 'cgau4', coif='cgau4')

In [ ]:
def visualize_wavelet(wavelet_name):
    """Отрисовывает сам вейвлет."""
    wavelet = pywt.Wavelet(wavelet_name)
    plt.figure(figsize=(14, 3))
    if wavelet.complex_cwt:
        phi, x = wavelet.wavefun(level=10)
        plt.plot(x, np.real(phi), label='Real')
        plt.plot(x, np.imag(phi), label='Imaginary')
        plt.title(f'Wavelet: {wavelet_name}')
        plt.legend()
    else:
        try:  # Обработка исключения, если у вейвлета нет psi
             phi, psi, x = wavelet.wavefun(level=10)
             plt.plot(x, psi)
             plt.title(f'Wavelet: {wavelet_name}')
        except ValueError:
             print(f"У вейвлета {wavelet_name} нет psi")

    plt.show()

def analyze_all_cryptos(decomposed_data, wavelet_names):
    """
    Выполняет CWT для всех криптовалют и выбранных вейвлетов.

    Args:
        decomposed_data (dict): Результат decompose_timeseries.
        wavelet_names (list): Список имён вейвлетов для анализа.

    Returns:
        dict: Словарь с результатами CWT для каждой криптовалюты и каждого вейвлета.
    """

    results = {}
    for ticker, data in decomposed_data.items():
        if data is not None:
            results[ticker] = {}
            residuals = data['residuals']
            for wavelet_name in wavelet_names:
                print(f"Анализ {ticker} с вейвлетом {wavelet_name}...")
                coef, frequencies = cwt_analysis(residuals, wavelet_name=wavelet_name)
                results[ticker][wavelet_name] = {
                    'coef': coef,
                    'frequencies': frequencies
                }
                visualize_cwt(residuals, coef, frequencies, wavelet_name, ticker)  # Визуализация скалограммы
                visualize_wavelet(wavelet_name) #Визуализация вейвлета
    return results

# Пример использования:

# Список вейвлетов для анализа
wavelet_names = ['cmor1.5-1.0', 'morl', 'gaus2', 'shan1-1.5', 'cgau4']

# Выполняем CWT для всех криптовалют и вейвлетов
cwt_results = analyze_all_cryptos(decomposed_crypto_data, wavelet_names)

# Теперь в cwt_results хранятся результаты CWT.
#  Доступ к результатам:
cwt_results['BTC-USD']['cmor1.5-1.0']['coef'] - вейвлет-коэффициенты для BTC-USD и cmor1.5-1.0
cwt_results['BTC-USD']['cmor1.5-1.0']['frequencies'] - соответствующие частоты.

# (Дальнейший анализ: Feature Engineering и машинное обучение)

In [ ]:
def visualize_global_wavelet_spectrum(coef, frequencies, ticker, wavelet_name):
    """Визуализирует средний вейвлет-спектр."""
    global_ws = (abs(coef) ** 2).mean(axis=1)  # Усреднение по времени (axis=1)

    plt.figure(figsize=(10, 4))
    plt.plot(frequencies, global_ws)
    plt.title(f'Global Wavelet Spectrum ({wavelet_name}) - {ticker}')
    plt.xlabel('Frequency (1/days)')
    plt.ylabel('Power')  # Или Amplitude^2
    plt.gca().invert_xaxis() # Чтобы частота уменьшалась слева направо
    plt.grid(True)
    plt.show()

# В analyze_all_cryptos, после visualize_cwt и visualize_wavelet:
visualize_global_wavelet_spectrum(coef, frequencies, ticker, wavelet_name)
# И добавить в results:
results[ticker][wavelet_name]['global_ws'] = global_ws

In [ ]:
from wcoherence import cwt, wct  # Импортируем

def visualize_wavelet_coherence(data1, data2, ticker1, ticker2, wavelet_name):
  """Визуализирует вейвлет-когерентность."""
  # CWT для обоих рядов (можно использовать уже рассчитанные ранее)
  coef1, freqs1 = cwt(data1, 1, 1, wavelet_name, 100) #num=100 масштабов
  coef2, freqs2 = cwt(data2, 1, 1, wavelet_name, 100)

  # Проверка совпадения частот
  if not np.allclose(freqs1, freqs2):
      raise ValueError("Частоты вейвлет-преобразований не совпадают!")

  # Расчет и визуализация когерентности
  wc, awc, coi, freq = wct(data1, data2, 1, wavelet_name, freqs=freqs1) #freqs подаем явно

  plt.figure(figsize=(12,6))
  im = plt.imshow(wc, extent=[0, len(data1), freqs1[-1], freqs1[0]], aspect='auto', cmap='jet', origin='upper')
  plt.colorbar(label="Coherence")
  plt.title(f"Wavelet Coherence ({wavelet_name}) - {ticker1} vs {ticker2}")
  plt.xlabel("Time (days)")
  plt.ylabel("Frequency (1/days)")
  plt.gca().invert_yaxis()
  #Отрисовка "конуса влияния"
  plt.fill_between(np.arange(len(data1)), coi, freqs1[-1], color='white', alpha=0.5)
  plt.show()

#Пример использования (внутри цикла по тикерам, например):
#  (Предполагаем, что у нас есть residuals для ticker1 и ticker2)
visualize_wavelet_coherence(residuals1, residuals2, ticker1, ticker2, 'cmor1.5-1.0')

In [ ]:
def visualize_phase_plot(coef, frequencies, ticker, wavelet_name):
    """Визуализирует фазовую диаграмму."""
    plt.figure(figsize=(14, 6))
    plt.imshow(np.angle(coef), extent=[0, len(data), frequencies[-1], frequencies[0]], aspect='auto',
               cmap='hsv', origin='upper')  # Используем цветовую карту 'hsv' для фазы
    plt.colorbar(label='Phase (radians)')
    plt.title(f'Phase Plot ({wavelet_name}) - {ticker}')
    plt.ylabel('Frequency (1/days)')
    plt.xlabel('Time (days)')
    plt.gca().invert_yaxis()
    plt.show()

#В analyze_all_cryptos:
visualize_phase_plot(coef, frequencies, ticker, wavelet_name)
# И добавить в results:
results[ticker][wavelet_name]['phase'] = np.angle(coef)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def visualize_3d_scalogram(coef, frequencies, ticker, wavelet_name):
  """Визуализирует 3D-скалограмму."""
  fig = plt.figure(figsize=(14, 8))
  ax = fig.add_subplot(111, projection='3d')
  X, Y = np.meshgrid(np.arange(coef.shape[1]), frequencies)
  Z = abs(coef)

  surf = ax.plot_surface(X, Y, Z, cmap='jet', linewidth=0, antialiased=False)

  ax.set_xlabel('Time (days)')
  ax.set_ylabel('Frequency (1/days)')
  ax.set_zlabel('Magnitude')
  ax.set_title(f'3D Scalogram ({wavelet_name}) - {ticker}')
  ax.view_init(elev=30, azim=120) # Настройка угла обзора
  fig.colorbar(surf, shrink=0.5, aspect=5)
  plt.gca().invert_yaxis()
  plt.show()

#В analyze_all_cryptos:
visualize_3d_scalogram(coef, frequencies, ticker, wavelet_name)